In [ ]:
# default_exp optuna

# Optuna: A hyperparameter optimization framework

> Optuna is an automatic hyperparameter optimization software framework, particularly designed for machine learning. It features an imperative, define-by-run style user API. Thanks to our define-by-run API, the code written with Optuna enjoys high modularity, and the user of Optuna can dynamically construct the search spaces for the hyperparameters.

In [ ]:
#export
from pathlib import Path
from fastcore.script import *
import joblib
from tsai.imports import *
from importlib import import_module
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# # hide
# from pathlib import Path
# from fastcore.script import *
# import joblib
# from tsai.imports import *
# from importlib import import_module
# import warnings
# warnings.filterwarnings("ignore")


# @call_parse
# def optuna_study(
#     config:             Param('Path to the study config file', str), 
#     study_type:         Param('Type of study', str)=None,
#     multivariate:       Param('Flag to show progress bars or not.', store_false)=True,
#     study_name:         Param("Study's name. If this argument is set to None, a unique name is generated automatically.", str)=None, 
#     seed:               Param('Seed for random number generator.', int)=None, 
#     search_space:       Param('Path to dictionary whose keys and values are a parameter name and the corresponding candidates of values', str)=None, 
#     direction:          Param('Direction of optimization.', str)='maximize',
#     n_trials:           Param('The number of trials.', int)=None, 
#     timeout:            Param('Stop study after the given number of second(s).', int)=None, 
#     gc_after_trial:     Param('Flag to determine whether to automatically run garbage collection after each trial.', store_true)=False,
#     show_progress_bar:  Param('Flag to show progress bars or not.', store_false)=True,
#     show_plots:         Param('Flag to show plots or not.', store_false)=True,
#     save:               Param('Flag to save study to disk or not.', store_false)=True,
#     path:               Param('Path where the study will be saved', str)='optuna', 
#     ):

#     try: import optuna
#     except ImportError: raise ImportError('You need to install optuna!') 

#     while True: 
#         if config[0] in "/ .": config = config.split(config[0], 1)[1]
#         else: break
#     if '/' in config and config.rsplit('/', 1)[0] not in sys.path: sys.path.append(config.rsplit('/', 1)[0])
#     m = import_file_as_module(config)
#     assert hasattr(m, 'objective'), f"there's no objective function in {config}"
#     objective = getattr(m, "objective")

#     if study_type is None or study_type.lower() == "bayesian": sampler = optuna.samplers.TPESampler(seed=seed, multivariate=multivariate)
#     elif study_type.lower() in ["gridsearch", "gridsampler"]: 
#         assert hasattr(m, 'search_space'), f"there's no search_space function in {search_space}"
#         search_space = getattr(m, 'search_space')
#         sampler = optuna.samplers.GridSampler(search_space=search_space)
#     elif study_type.lower() in ["randomsearch", "randomsampler"]: sampler = optuna.samplers.RandomSampler(seed=seed)
    
#     try: 
#         study = optuna.create_study(sampler=sampler, study_name=study_name, direction=direction)
#         study.optimize(objective, n_trials=n_trials, timeout=timeout, gc_after_trial=gc_after_trial, show_progress_bar=show_progress_bar)
        
#     except KeyboardInterrupt:
#         pass

#     if save: 
#         full_path = Path(path)/f'{study.study_name}.pkl'
#         full_path.parent.mkdir(parents=True, exist_ok=True)
#         joblib.dump(study, full_path)
#         print(f'\nOptuna study saved to {full_path}')
#         print(f"To reload the study run: study = joblib.load('{full_path}')")

#     if show_plots and len(study.trials) > 1: 
#         try: display(optuna.visualization.plot_optimization_history(study))
#         except: pass
#         try: display(optuna.visualization.plot_param_importances(study))
#         except: pass
#         try: display(optuna.visualization.plot_slice(study))
#         except: pass
#         try: display(optuna.visualization.plot_parallel_coordinate(study))
#         except: pass
    
#     try: 
#         print(f"\nStudy stats   : ")
#         print(f"===============")
#         print(f"Study name    : {study.study_name}")
#         print(f"  n_trials    : {len(study.trials)}")
#         print(f"Best trial    :")
#         trial = study.best_trial
#         print(f"  value       : {trial.value}")
#         print(f"  best_params = {trial.params}\n")
#     except:
#         print('No trials are completed yet.')
#     return study

In [ ]:
#export
def retrieve_value(var):
    callers_local_vars = inspect.currentframe().f_back.f_globals.items()
    return [var_val for var_name, var_val in callers_local_vars if var_name is var]

def get_current_config(multi_value_dict):
    keys = list(multi_value_dict.keys())
    values = []
    for i in range(len(keys)): values.append(retrieve_value(keys[i])[0])
    return AttrDict({k:v for k,v in zip(keys, values)})

In [ ]:
multi_config = {'a': [1,2,3,4], 'b':['cat', 'dog']}
a = 3
b = "cat"
get_current_config(multi_config)

- a: 3
- b: cat

In [ ]:
#export
from pathlib import Path
from fastcore.script import *
import joblib
from tsai.imports import *
from importlib import import_module
import warnings
warnings.filterwarnings("ignore")


def run_optuna_study(objective, study_type=None, multivariate=True, search_space=None, seed=None, sampler=None, pruner=None, study_name=None, 
                     direction='maximize', load_if_exists=False, n_trials=None, timeout=None, gc_after_trial=False, show_progress_bar=True, save_study=True, 
                     path='optuna', show_plots=True):
    r"""Creates and runs an optuna study.

    Args: 
        objective:          A callable that implements objective function.
        study_type:         Type of study selected (bayesian, gridsearch, randomsearch). Based on this a sampler will be build if sampler is None. 
                            If a sampler is passed, this has no effect.
        multivariate:       If this is True, the multivariate TPE is used when suggesting parameters. The multivariate TPE is reported to outperform 
                            the independent TPE.
        search_space:       Search space required when running a gridsearch (if you don't pass a sampler).
        seed:               Fixed seed used by samplers.
        sampler:            A sampler object that implements background algorithm for value suggestion. If None is specified, TPESampler is used during 
                            single-objective optimization and NSGAIISampler during multi-objective optimization. See also samplers.
        pruner:             A pruner object that decides early stopping of unpromising trials. If None is specified, MedianPruner is used as the default. 
                            See also pruners.
        study_name:         Study’s name. If this argument is set to None, a unique name is generated automatically.
        direction:          A sequence of directions during multi-objective optimization.
        n_trials:           The number of trials. If this argument is set to None, there is no limitation on the number of trials. If timeout is also set to 
                            None, the study continues to create trials until it receives a termination signal such as Ctrl+C or SIGTERM.
        timeout:            Stop study after the given number of second(s). If this argument is set to None, the study is executed without time limitation. 
                            If n_trials is also set to None, the study continues to create trials until it receives a termination signal such as 
                            Ctrl+C or SIGTERM.
        gc_after_trial:     Flag to execute garbage collection at the end of each trial. By default, garbage collection is enabled, just in case. 
                            You can turn it off with this argument if memory is safely managed in your objective function.
        show_progress_bar:  Flag to show progress bars or not. To disable progress bar, set this False.
        save_study:         Save your study when finished/ interrupted.
        path:               Folder where the study will be saved.
        show_plots:         Flag to control whether plots are shown at the end of the study.
    """
    try: import optuna
    except ImportError: raise ImportError('You need to install optuna!')

    # Sampler
    if sampler is None:
        if study_type is None or study_type.lower() == "bayesian": sampler = optuna.samplers.TPESampler(seed=seed, multivariate=multivariate)
        elif study_type.lower() in ["gridsearch", "gridsampler"]:
            assert search_space, f"you need to pass a search_space dict to run a gridsearch"
            sampler = optuna.samplers.GridSampler(search_space)
        elif study_type.lower() in ["randomsearch", "randomsampler"]: sampler = optuna.samplers.RandomSampler(seed=seed)
    assert sampler, "you need to select a study type (bayesian, gridsampler, randomsampler) or pass a sampler"

    # Study
    try:
        study = optuna.create_study(sampler=sampler, pruner=pruner, study_name=study_name, direction=direction)
        study.optimize(objective, n_trials=n_trials, timeout=timeout, gc_after_trial=gc_after_trial, show_progress_bar=show_progress_bar)
    except KeyboardInterrupt:
        pass

    # Save
    if save_study:
        full_path = Path(path)/f'{study.study_name}.pkl'
        full_path.parent.mkdir(parents=True, exist_ok=True)
        joblib.dump(study, full_path)
        print(f'\nOptuna study saved to {full_path}')
        print(f"To reload the study run: study = joblib.load('{full_path}')")

    # Plots
    if show_plots and len(study.trials) > 1:
        try: display(optuna.visualization.plot_optimization_history(study))
        except: pass
        try: display(optuna.visualization.plot_param_importances(study))
        except: pass
        try: display(optuna.visualization.plot_slice(study))
        except: pass
        try: display(optuna.visualization.plot_parallel_coordinate(study))
        except: pass

    # Study stats
    try:
        print(f"\nStudy stats   : ")
        print(f"===============")
        print(f"Study name    : {study.study_name}")
        print(f"  n_trials    : {len(study.trials)}")
        print(f"Best trial    :")
        trial = study.best_trial
        print(f"  value       : {trial.value}")
        print(f"  best_params = {trial.params}\n")
    except:
        print('No trials are completed yet.')
    return study

In [ ]:
#hide
from tsai.imports import create_scripts
from tsai.export import get_nb_name
nb_name = get_nb_name()
create_scripts(nb_name);

Converted 200_optuna.ipynb.


Correct conversion! 😃
Total time elapsed 0.303 s
Wednesday 29/09/21 12:55:07 CEST
